In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"D:\gpu_libraries\deep_learning_with_tensorflow\data\archive\diabetes.csv")
df.sample(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
100,1,163,72,0,0,39.0,1.222,33,1
740,11,120,80,37,150,42.3,0.785,48,1
163,2,100,64,23,0,29.7,0.368,21,0
337,5,115,76,0,0,31.2,0.343,44,1
686,3,130,64,0,0,23.1,0.314,22,0
496,5,110,68,0,0,26.0,0.292,30,0
472,0,119,66,27,0,38.8,0.259,22,0
657,1,120,80,48,200,38.9,1.162,41,0
244,2,146,76,35,194,38.2,0.329,29,0
38,2,90,68,42,0,38.2,0.503,27,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X= df.drop(columns = ['Outcome'],axis = 1)
Y = df['Outcome']

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 48.1 KB


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
X_trans = scaler.fit_transform(X)


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_trans, Y, random_state = 42, test_size = 0.15)

In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [10]:
model = Sequential()
model.add(Dense(32,activation = "relu", input_dim = 8))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer = 'Adam',loss = 'binary_crossentropy', metrics = ['accuracy'] )

In [11]:
model.fit(X_train,Y_train,epochs = 100,batch_size = 32,validation_data = (X_test,Y_test))

Epoch 1/100
21/21 [==============================] - 2s 10ms/step - loss: 0.7300 - accuracy: 0.4709 - val_loss: 0.6847 - val_accuracy: 0.5172
Epoch 2/100
21/21 [==============================] - 0s 4ms/step - loss: 0.6522 - accuracy: 0.6181 - val_loss: 0.6272 - val_accuracy: 0.6638
Epoch 3/100
21/21 [==============================] - 0s 4ms/step - loss: 0.6009 - accuracy: 0.6840 - val_loss: 0.5912 - val_accuracy: 0.6552
Epoch 4/100
21/21 [==============================] - 0s 4ms/step - loss: 0.5670 - accuracy: 0.7071 - val_loss: 0.5654 - val_accuracy: 0.6897
Epoch 5/100
21/21 [==============================] - 0s 4ms/step - loss: 0.5429 - accuracy: 0.7193 - val_loss: 0.5481 - val_accuracy: 0.7328
Epoch 6/100
21/21 [==============================] - 0s 4ms/step - loss: 0.5250 - accuracy: 0.7255 - val_loss: 0.5334 - val_accuracy: 0.7414
Epoch 7/100
21/21 [==============================] - 0s 4ms/step - loss: 0.5120 - accuracy: 0.7362 - val_loss: 0.5219 - val_accuracy: 0.7414
Epoch 8/100


# Defining hidden layers and all other parameter can be atumatically selected according to best result and for this we can use keras tunner

1. how to select appropriate optimizer
2. No of nodes in a layer
3. how to select no.of layers
4. All in one model

In [12]:
import keras_tuner as kt

In [13]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units',min_value= 8, max_value = 128, step = 1) # for finding best number of nodes in this layers
    model.add(Dense(units = units,activation = 'relu',input_dim = 8))
    model.add(Dense(1,activation = 'sigmoid'))
    op = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adagrad','adadelta']) # for finding best optimizer
    model.compile(optimizer = op, loss = 'binary_crossentropy',metrics = ['accuracy'])
    return model
    

In [14]:
# now create tunner object
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy',max_trials = 5,directory = 'Tunner',project_name = 'Tunner_practice2')

In [15]:
tuner.search(X_train,Y_train,epochs = 5, validation_data = (X_test,Y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.6637930870056152

Best val_accuracy So Far: 0.732758641242981
Total elapsed time: 00h 00m 06s


In [16]:
tuner.get_best_hyperparameters()[0].values

{'units': 60, 'optimizer': 'adam'}

In [17]:
# extraction best model
model = tuner.get_best_models(num_models = 1)[0]


In [18]:
model.fit(X_train,Y_train, epochs = 100, batch_size = 32, initial_epoch= 6)

Epoch 7/100
21/21 [==============================] - 0s 3ms/step - loss: 0.5033 - accuracy: 0.7730
Epoch 8/100
21/21 [==============================] - 0s 4ms/step - loss: 0.4891 - accuracy: 0.7776
Epoch 9/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4782 - accuracy: 0.7776
Epoch 10/100
21/21 [==============================] - 0s 4ms/step - loss: 0.4708 - accuracy: 0.7776
Epoch 11/100
21/21 [==============================] - 0s 4ms/step - loss: 0.4650 - accuracy: 0.7837
Epoch 12/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4603 - accuracy: 0.7853
Epoch 13/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4561 - accuracy: 0.7822
Epoch 14/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4530 - accuracy: 0.7822
Epoch 15/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4507 - accuracy: 0.7837
Epoch 16/100
21/21 [==============================] - 0s 3ms/step - loss: 0.4484 - accuracy: 0.7837
Epo

In [19]:
#3 number of layers
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation = 'relu', input_dim = 8))
    for i in range(hp.Int('units',min_value= 1, max_value = 10, step = 1)):
        model.add(Dense(units = 32,activation = 'relu'))
    model.add(Dense(1,activation = 'sigmoid'))
    op = hp.Choice('optimizer',values = ['rmsprop']) # for finding best optimizer
    model.compile(optimizer = op, loss = 'binary_crossentropy',metrics = ['accuracy'])
    return model

In [20]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy',max_trials = 3,directory = 'Tunner',project_name = 'Tunner_practice')

In [21]:
tuner.search(X_train,Y_train,epochs = 5, validation_data = (X_test,Y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.767241358757019

Best val_accuracy So Far: 0.7758620977401733
Total elapsed time: 00h 00m 06s


In [22]:
tuner.get_best_hyperparameters()[0].values

{'units': 7, 'optimizer': 'rmsprop'}

In [23]:
model = tuner.get_best_models(num_models = 1)[0]

In [24]:
model.fit(X_train,Y_train, epochs = 100, batch_size = 32, initial_epoch= 6)

Epoch 7/100
21/21 [==============================] - 1s 7ms/step - loss: 0.4513 - accuracy: 0.7853
Epoch 8/100
21/21 [==============================] - 0s 6ms/step - loss: 0.4368 - accuracy: 0.7960
Epoch 9/100
21/21 [==============================] - 0s 7ms/step - loss: 0.4277 - accuracy: 0.7945
Epoch 10/100
21/21 [==============================] - 0s 7ms/step - loss: 0.4174 - accuracy: 0.8098
Epoch 11/100
21/21 [==============================] - 0s 7ms/step - loss: 0.4084 - accuracy: 0.8190
Epoch 12/100
21/21 [==============================] - 0s 7ms/step - loss: 0.3989 - accuracy: 0.8160
Epoch 13/100
21/21 [==============================] - 0s 6ms/step - loss: 0.3940 - accuracy: 0.8313
Epoch 14/100
21/21 [==============================] - 0s 6ms/step - loss: 0.3843 - accuracy: 0.8221
Epoch 15/100
21/21 [==============================] - 0s 6ms/step - loss: 0.3730 - accuracy: 0.8405
Epoch 16/100
21/21 [==============================] - 0s 7ms/step - loss: 0.3665 - accuracy: 0.8282
Epo

In [38]:
from keras.layers import Dropout

In [39]:
def build_model(hp):
    model=Sequential()
    counter = 0
    
    for i in range(hp.Int('num_layers',min_value = 1, max_value = 10, step = 1)):
        if counter == 0:
            model.add(Dense(
                hp.Int('units'+str(i),min_value = 8, max_value = 128,step = 8),
                activation = hp.Choice('activation'+str(i),values= ['relu','tanh','sigmoid']),
                input_dim = 8
                )
                )
            model.add(Dropout(hp.Choice('dropout'+str(i), values =[0.1,0.2,0.3,0.4])))
        else:
            model.add(Dense(
                hp.Int('units'+str(i),min_value = 8, max_value = 128,step = 8),
                activation = hp.Choice('activation'+str(i),values= ['relu','tanh','sigmoid'])
                
                )
                )
            model.add(Dropout(hp.Choice('dropout'+str(i), values =[0.1,0.2,0.3,0.4])))
        counter+=1 

    model.add(Dense(1,activation = 'sigmoid'))
    op = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adagrad','adadelta']) # for finding best optimizer
    model.compile(optimizer = op, loss = 'binary_crossentropy',metrics = ['accuracy'])
    return model


In [41]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy',max_trials = 3,directory = 'Tunner',project_name = 'Tunner_practice9')

In [42]:
tuner.search(X_train,Y_train,epochs = 5, validation_data = (X_test,Y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.6551724076271057

Best val_accuracy So Far: 0.732758641242981
Total elapsed time: 00h 00m 06s


In [43]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 56,
 'activation0': 'relu',
 'dropout0': 0.3,
 'optimizer': 'rmsprop',
 'units1': 96,
 'activation1': 'tanh',
 'dropout1': 0.4,
 'units2': 128,
 'activation2': 'sigmoid',
 'dropout2': 0.4,
 'units3': 56,
 'activation3': 'tanh',
 'dropout3': 0.3,
 'units4': 80,
 'activation4': 'tanh',
 'dropout4': 0.1,
 'units5': 16,
 'activation5': 'relu',
 'dropout5': 0.1}

In [44]:
model = tuner.get_best_models(num_models = 1)[0]

In [45]:
model.fit(X_train,Y_train, epochs = 100, batch_size = 32, initial_epoch= 6,validation_data= (X_test,Y_test))

Epoch 7/100
21/21 [==============================] - 1s 10ms/step - loss: 0.5225 - accuracy: 0.7485 - val_loss: 0.5322 - val_accuracy: 0.7500
Epoch 8/100
21/21 [==============================] - 0s 5ms/step - loss: 0.5122 - accuracy: 0.7393 - val_loss: 0.5219 - val_accuracy: 0.7414
Epoch 9/100
21/21 [==============================] - 0s 6ms/step - loss: 0.4982 - accuracy: 0.7638 - val_loss: 0.5146 - val_accuracy: 0.7414
Epoch 10/100
21/21 [==============================] - 0s 5ms/step - loss: 0.4979 - accuracy: 0.7577 - val_loss: 0.5096 - val_accuracy: 0.7414
Epoch 11/100
21/21 [==============================] - 0s 5ms/step - loss: 0.4861 - accuracy: 0.7592 - val_loss: 0.5054 - val_accuracy: 0.7586
Epoch 12/100
21/21 [==============================] - 0s 5ms/step - loss: 0.4924 - accuracy: 0.7761 - val_loss: 0.5020 - val_accuracy: 0.7586
Epoch 13/100
21/21 [==============================] - 0s 5ms/step - loss: 0.4866 - accuracy: 0.7684 - val_loss: 0.4998 - val_accuracy: 0.7586
Epoch 14